In [2]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import os
import tqdm

Information about data: 
- CAGE-train ("data/CAGE-train"): includes information about gene location and expression. X1 and X2 has train/val info/y, X3 only has test info that we should test on (only testing on chr1). 
- DNase-seq: information on chromatin accessibility (reads only in open chromatin)
- bed, bigwig: genomic regions and associated annotations 
- do not use H3K9me3, distribution too different among X1, X2, X3 


## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [6]:
# TODO: 
# Load your feature (bed and/or bigwig and/or fasta) and target files (tsv) here.
# Decide which features to use for training. Feel free to process them however you need.

# NOTE: 
# bed and bigwig files contain signals of all chromosomes (including sex chromosomes).
# Training and validation split based on chromosomes has been done for you. 
# However, you can resplit the data in any way you want.

current_path = os.getcwd()
path_data = current_path + "/data/"
path_test = current_path + "/data/CAGE-train/X3_test_info.tsv"   
test_genes = pd.read_csv(path_test, sep='\t')

# load bed files from separate files in a loop and store them in a list
bedfiles = ['DNase-bed', 'H3K4me1-bed', 'H3K4me3-bed', 'H3K9me3-bed', 'H3K27ac-bed', 'H3K27me3-bed', 'H3K36me3-bed']
bed_files_x1 = {}
bed_files_x2 = {}
bed_files_x3 = {}

for i in bedfiles:
    bed_files_x1[str(i)] = pd.read_csv(path_data + i + '/X1.bed', sep='\t', header=None)
    bed_files_x2[str(i)] = pd.read_csv(path_data + i + '/X2.bed', sep='\t', header=None)
    bed_files_x3[str(i)] = pd.read_csv(path_data + i + '/X3.bed', sep='\t', header=None)

# loading gene expression 
gex_path = 'CAGE-train'
x1_train_info = pd.read_csv(path_data + gex_path + '/X1_train_info.tsv', sep='\t')
x1_train_y = pd.read_csv(path_data + gex_path + '/X1_train_y.tsv', sep='\t')

x1_val_info = pd.read_csv(path_data + gex_path + '/X1_val_info.tsv', sep='\t')
x1_val_y = pd.read_csv(path_data + gex_path + '/X1_val_y.tsv', sep='\t')

x2_train_info = pd.read_csv(path_data + gex_path + '/X2_train_info.tsv', sep='\t')
x2_train_y = pd.read_csv(path_data + gex_path + '/X2_train_y.tsv', sep='\t')

x2_val_info = pd.read_csv(path_data + gex_path + '/X2_val_info.tsv', sep='\t')
x2_val_y = pd.read_csv(path_data + gex_path + '/X2_val_y.tsv', sep='\t')



In [161]:
reach = 5000

# extracting features from bed files and gene info data
def extract_features(input_bed_files, gene_info):
    # iterate through rows in gene_info 
    # return pandas dataframe with features

    # create empty dataframe
    features = pd.DataFrame()
    for index, row in gene_info.iterrows():
        chr = row['chr']
        gene_length = row['gene_end'] - row['gene_start']
        tss_start = row['TSS_start'] - reach
        tss_end = row['TSS_start'] + reach
        # loop through bed files
        for j in bedfiles:
            print(j)
            bed_files = input_bed_files[j]
            # filter bed files by chromosome
            bed_files_filt = bed_files.loc[bed_files[0] == chr]
            # print(bed_files_filt)
            # select for rows that are within reach of TSS
            bed_files_final = bed_files_filt[(bed_files_filt[1] >= tss_start) & (bed_files_filt[2] <= tss_end)]
            # if bed_files_final not empty
            if bed_files_final.size > 0:
                if j == 'DNase-bed':
                    score = bed_files_final[6]
                else:
                    # print(score)
                    score = bed_files_final[4]
                # peaks location 
                peak_start = bed_files_final[1]
                peak_end = bed_files_final[2]

                # sum of score
                sum_score = score.sum()
                # average score
                avg_score = score.mean()
                # max score
                max_score = score.max()
                # number of peaks
                num_peaks = len(score)

                # distance of tss midpoint to nearest peak midpoint
                peak_midpoint = (peak_start + peak_end) / 2
                tss_midpoint = (tss_start + tss_end) / 2
                if num_peaks == 0:
                    # arbtiary large distance if no peaks
                    dist_to_peak = 2 * reach
                else:
                    # find index of peak which has midpoint closest to tss midpoint
                    closest_peak = np.argmin(np.abs(peak_midpoint - tss_midpoint))
                    # distance of tss midpoint to nearest peak midpoint
                    dist_to_peak = np.abs(peak_midpoint[closest_peak] - tss_midpoint)
            else:
                sum_score = 0
                avg_score = 0
                max_score = 0
                num_peaks = 0
                dist_to_peak = 2 * reach
        features = features.append({'gene_length': gene_length, 'sum_score': sum_score, 'avg_score': avg_score, 'max_score': max_score, 
                                        'num_peaks': num_peaks, 'dist_to_peak': dist_to_peak}, ignore_index=True)
    features.set_index(gene_info['gene_id'], inplace=True)
    return features


In [162]:
# extract features for each dataset
x1_train_features = extract_features(bed_files_x1, x1_train_info)

# print(bed_files_x1['DNase-bed'])

DNase-bed
H3K4me1-bed


KeyError: 2

In [101]:
# define function to subtract and add to tss region
def tss_region(tss, start, end):
    tss['TSS_start'] = tss['TSS_start'] - start
    tss['TSS_end'] = tss['TSS_end'] + end
    return tss

# define a function to loop through bed files and extract signal sum in TSS region
def extract_signal(bed_files, tss, features):
    for i in tqdm.tqdm(bedfiles, desc= "looping through bedfiles"):
        bed = bed_files[i]
        chr = bed[0]
        start = bed[1]
        end = bed[2]
        
        if i == 'DNase-bed':
            signal = bed[6]
        else:
            signal = bed[4]
        for j in tss.index:
            if any((chr == tss.loc[j, 'chr']) & (start >= tss.loc[j, 'TSS_start']) & (end <= tss.loc[j, 'TSS_end'])):
                tss_signal = signal[(chr == tss.loc[j, 'chr']) & (start >= tss.loc[j, 'TSS_start']) & (end <= tss.loc[j, 'TSS_end'])].sum()
            else:
                tss_signal = 0
            features.loc[j, i] = tss_signal
        print("finished " + i)
        print(features.loc[:,i])
    features = pd.DataFrame(features)
    features.set_index(tss.index)
    return features

In [102]:
# define tss region
tss_train = pd.DataFrame(x1_train_info[['gene_name', 'chr', 'TSS_start', 'TSS_end']])
tss_train = tss_train.set_index('gene_name')
tss_val = pd.DataFrame(x1_val_info[['gene_name', 'chr', 'TSS_start', 'TSS_end']])
tss_val = tss_val.set_index('gene_name')
tss_train = tss_region(tss_train, 5000, 5000)
tss_val = tss_region(tss_val, 5000, 5000)

tss_x3 = test_genes[['gene_name', 'chr', 'TSS_start', 'TSS_end']]
tss_x3 = tss_x3.set_index('gene_name')
tss_x3 = tss_region(tss_x3, 5000, 5000)


In [103]:
# extract signal for x1 train, val, x2 train, val
features_x1_train = pd.DataFrame()
features_x1_val = pd.DataFrame()
features_x2_train = pd.DataFrame()
features_x2_val = pd.DataFrame()


features_x1_train = extract_signal(bed_files_x1, tss_train, features_x1_train)
features_x1_val = extract_signal(bed_files_x1, tss_val, features_x1_val)
features_x2_train = extract_signal(bed_files_x2, tss_train, features_x2_train)
features_x2_val = extract_signal(bed_files_x2, tss_val, features_x2_val)

looping through bedfiles:   0%|          | 0/7 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# extract signal for x3
features_x3_test = pd.DataFrame()
features_x3_test = extract_signal(bed_files_x3, tss_x3, features_x3_test)

## Work Package 1.2 - Model Building

In [ ]:
# TODO: 
# Select the best model to predict gene expression from the obtained features in WP 1.1.

# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [ ]:
# TODO:
# Using the model trained in WP 1.2, make predictions on the test data (chr 1 of cell line X3).
# Store predictions in a variable called "pred" which is a numpy array.

pred = None
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'path/to/save/output/file'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "LastName_FirstName_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes['gex_predicted'] = pred.tolist()
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)